In [1]:
import splinter
from bs4 import BeautifulSoup
import time

In [2]:
url = 'https://www.drivethrurpg.com/en/browse?ruleSystem=44826&page='
outfilename = 'drivethrurpg-chaosium.json'

In [3]:
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# #browser = splinter.Browser()
# browser = selenium.webdriver.Chrome()
# browser.get(url)
# time.sleep(3)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(2)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# soup = BeautifulSoup(browser.page_source, parser="html")
# browser.quit()

In [4]:
def match_attr(tag):
    return tag.name == "ul" and tag['class'] == ['list-clean'] and not ("_ngcontent-serverapp-c1510817403" in tag.attrs)

In [5]:
def match_obs(tag):
    return tag and tag.name == "obs-result-list" and ("_nghost-serverapp-c1510817403" in tag.attrs)

In [6]:
urls = []
browser = splinter.Browser()
for page in range(1,4):
    browser.visit(f"{url}{page}")
    time.sleep(3)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(browser.html, parser="html")
    obs = soup.find(match_obs)
    ul = obs.find("ul", class_="list-clean")
    lis = ul.find_all("li")
    for li in lis:
        a = li.find("a")
        if 'href' in a.attrs:
            urls.append("https://www.drivethrurpg.com" + a['href'])
            print(a['href'])
browser.quit()

/en/product/370454/runequest-starter-set
/en/product/475104/going-underground-rivers-of-london-rpg-scenario
/en/product/469816/call-of-cthulhu-arkham
/en/product/260810/call-of-cthulhu-starter-set
/en/product/431991/basic-roleplaying-universal-game-engine
/en/product/462154/heinrich-s-call-of-cthulhu-guide-to-carcosa
/en/product/476836/alone-against-nyarlathotep
/en/product/480447/echoes-in-the-mist-a-regency-cthulhu-story-guide
/en/product/476571/kane-s-tone
/en/product/446091/japan-empire-of-shadows-a-call-of-cthulhu-sourcebook-for-1920s-imperial-japan
/en/product/476224/power-outage
/en/product/391974/viral-a-modern-call-of-cthulhu-scenario
/en/product/461171/alone-against-the-static
/en/product/480227/the-million-flavoured-ones
/en/product/418884/the-comae-engine
/en/product/476312/in-darkness-we-wait
/en/product/385006/korean
/en/product/468943/book-of-schemes
/en/product/191475/mythras
/en/product/24384/basic-roleplaying
/en/product/386943/korean-the-world-of-cliche
/en/product/3

In [8]:
browser = splinter.Browser()
titles = []
for page in urls:
    print(f"visiting {page}")
    download = True
    while download:
        browser.visit(page)
        soup2 = BeautifulSoup(browser.html, parser="html")
        download = soup2.find("h2") == None
        time.sleep(2)
    title = soup2.find("h2").text.strip()
    page_count = None
    try:
        page_count = int(soup2.find("div", class_="product-detail-tile-4").find_all("p")[2].text.strip())
    except ValueError:
        print("page count not found")
    products = soup2.find("obs-product-format-options").find_all("obs-product-format-option-selection-tile")
    product_dicts = []
    for product in products:
        product_name = product.find("p", class_="product-tile-text-highlight").text.strip()
        price = None
        try:
            price = float(product.find_all("p", class_="product-tile-text-highlight")[-1].text.strip().replace("$",""))
        except ValueError:
            print(f"no price found for {title}:{product_name}")
        softcover = "oftcover" in product_name
        hardcover = "ardcover" in product_name
        premium = "remium" in product_name
        pdf = "PDF" in product_name or "pdf" in product_name
        color = "olor" in product_name
        product_dicts.append({
            "product":product_name, 
            "price":price, 
            "softcover":softcover,
            "hardcover":hardcover,
            "premium":premium,
            "pdf":pdf,
            "color":color
        })
    published_date = soup2.find("div", class_="product-detail-tile-5").find("p", class_="u-text-bold").text.strip()
    format = soup2.find("p", attrs={"data-codeid":"format"}).parent.next_sibling.find('p').text.strip()
    author_block = soup2.find("obs-publisher-or-creators", listonly="authors")
    authors = []
    for a in author_block.find_all("a"):
        authors.append(a.text.strip().replace(",", ""))
    artist_block = soup2.find("obs-publisher-or-creators", listonly="artists")
    artists = []
    for a in artist_block.find_all("a"):
        artists.append(a.text.strip().replace(",", ""))
    system = soup2.find("p", attrs={"data-codeid":"ruleSystem"}).parent.next_sibling.find("p").text.strip()
    modified_date = soup2.find("p", attrs={"data-codeid":"dateModified"}).parent.next_sibling.find("p").text.strip()
    titles.append({
        "title":title,
        "url":page,
        "page_count":page_count,
        "products":product_dicts,
        "published_date":published_date,
        "format":format,
        "authors":authors,
        "artists":artists,
        "system":system,
        "modified_date":modified_date
    })
browser.quit()
titles

visiting https://www.drivethrurpg.com/en/product/370454/runequest-starter-set
visiting https://www.drivethrurpg.com/en/product/475104/going-underground-rivers-of-london-rpg-scenario
visiting https://www.drivethrurpg.com/en/product/469816/call-of-cthulhu-arkham
visiting https://www.drivethrurpg.com/en/product/260810/call-of-cthulhu-starter-set
visiting https://www.drivethrurpg.com/en/product/431991/basic-roleplaying-universal-game-engine
visiting https://www.drivethrurpg.com/en/product/462154/heinrich-s-call-of-cthulhu-guide-to-carcosa
visiting https://www.drivethrurpg.com/en/product/476836/alone-against-nyarlathotep
visiting https://www.drivethrurpg.com/en/product/480447/echoes-in-the-mist-a-regency-cthulhu-story-guide
visiting https://www.drivethrurpg.com/en/product/476571/kane-s-tone
visiting https://www.drivethrurpg.com/en/product/446091/japan-empire-of-shadows-a-call-of-cthulhu-sourcebook-for-1920s-imperial-japan
visiting https://www.drivethrurpg.com/en/product/476224/power-outage


[{'title': 'RuneQuest Starter Set',
  'url': 'https://www.drivethrurpg.com/en/product/370454/runequest-starter-set',
  'page_count': 180,
  'products': [{'product': 'PDF',
    'price': 14.99,
    'softcover': False,
    'hardcover': False,
    'premium': False,
    'pdf': True,
    'color': False}],
  'published_date': '11/10/21',
  'format': 'PDF',
  'authors': ['Greg Stafford', 'Jeff Richard', 'Jason Durall'],
  'artists': ['Ossi Hiekkala',
   'Andrey Fetisov',
   'Jon Hodgson',
   'Kalin Kadiev',
   'Roman Kisyov',
   'Michelle Lockamy',
   'Jan Pospisil',
   'Scott Purdy',
   'Naomi Robinson',
   'Simon Roy',
   'Chris Waller',
   'Jon Hodgson',
   'Scott Purdy',
   'Roman Kisyov',
   'Dimitrina Angelska'],
  'system': 'RuneQuest, BRP (Basic Roleplaying)',
  'modified_date': 'Nov 9, 2021'},
 {'title': 'Going Underground - Rivers of London RPG Scenario',
  'url': 'https://www.drivethrurpg.com/en/product/475104/going-underground-rivers-of-london-rpg-scenario',
  'page_count': 40,
  '

In [9]:
import json
with open(outfilename, "w", encoding="utf-8") as out:
    json.dump(titles, out, ensure_ascii=False, indent='\t')